In [97]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
plt.rc('font', family='Verdana')

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import GridSearchCV

In [103]:
data_dummies = DataFrame.from_csv('data/Credit_OTP_processed_dummies.csv', encoding='utf-8')
y = data_dummies.loc[:, 'TARGET_1']
data_dummies.drop(['TARGET_0', 'TARGET_1'], axis=1, inplace=True)
X = data_dummies.loc[:, 'AGE':'GEN_PHONE_FL_1']
logreg_fullset = LogisticRegression().fit(X, y)
forest_fullset = RandomForestClassifier(n_estimators=500, max_depth=12, random_state=152, n_jobs=-1).fit(X, y)

In [104]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: TARGET_1, dtype: int64

In [107]:
forest_fullset.score(X, y)

0.88399132891020171

In [108]:
roc_auc_score(y, logreg_fullset.predict_proba(X)[:, 1])

0.68757661291849137

In [98]:
data = DataFrame.from_csv('data/Credit_OTP_processed_dummies.csv', encoding='utf-8')
data.head()

,AGE,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,CREDIT,TERM,FST_PAYMENT,FACT_LIVING_TERM,WORK_TIME,LOAN_NUM_TOTAL,...,REG_PHONE_FL_1,GEN_PHONE_FL_0,GEN_PHONE_FL_1,AGE_CATEGORY_1,AGE_CATEGORY_2,AGE_CATEGORY_3,AGE_CATEGORY_4,AGE_CATEGORY_5,WITHOUT_DELAY_0,WITHOUT_DELAY_1
0,49,5000.0,253,250,8000.00,6,8650.0,220,18.0,1,...,0,0,1,0,0,1,0,0,0,1
1,32,12000.0,127,127,21650.00,6,4000.0,137,97.0,1,...,0,0,1,0,1,0,0,0,0,1
2,52,9000.0,202,203,33126.00,12,4000.0,251,84.0,2,...,0,0,1,0,0,0,1,0,1,0
3,39,25000.0,407,404,8491.82,6,5000.0,36,168.0,1,...,1,0,1,0,1,0,0,0,0,1
4,30,12000.0,127,127,21990.00,12,4000.0,83,101.0,2,...,0,0,1,1,0,0,0,0,0,1


In [99]:
data.shape

(15223, 292)

In [100]:
X = data.drop('TARGET', axis=1)
y = data['TARGET']

ValueError: labels ['TARGET'] not contained in axis

In [ ]:
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = [5, 10, 25, 50, 100, 150, 200, 250, 300]

for ntrees in trees_grid:
    rf = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in strat.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        rf.fit(X_train, y_train)
        temp_train_acc.append(rf.score(X_train, y_train))
        temp_test_acc.append(rf.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Наилучшее значение правильности перекрестной проверки равно {:.2f}% и достигается при {} деревьях".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

In [ ]:
plt.style.use('ggplot')

fig, ax = plt.subplots(figsize=(16, 10))
ax.plot(trees_grid, train_acc.mean(axis=1), alpha=0.5, color='blue', 
        label='обучение')
ax.plot(trees_grid, test_acc.mean(axis=1), alpha=0.5, color='red', 
        label='перекрестная проверка')
ax.fill_between(trees_grid, test_acc.mean(axis=1) - test_acc.std(axis=1), test_acc.mean(axis=1) + test_acc.std(axis=1), color='#888888', alpha=0.4)
ax.fill_between(trees_grid, test_acc.mean(axis=1) - 2*test_acc.std(axis=1), test_acc.mean(axis=1) + 2*test_acc.std(axis=1), color='#888888', alpha=0.2)
ax.legend(loc='best')
ax.set_ylim([0.85,1.01])
ax.set_ylabel("Правильность")
ax.set_xlabel("Количество деревьев")

In [ ]:
n_trees = 75

In [ ]:
strat = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

X_trainval, X_test, y_trainval, y_test = train_test_split(data.drop('TARGET', axis=1), data['TARGET'],
                                                          test_size=0.3, random_state=42)
print("Размер обучающей выборки: {} размер тестовой выборки: {}".format(X_trainval.shape[0], X_test.shape[0]))

best_score = 0

for max_depth in [10, 12, 14, 16, 18, 20, 22]:
    for max_features in [3, 6, 9, 12, 15, 18, 21]:
        # для каждой комбинации параметров обучаем случайный лес
        rf = RandomForestClassifier(max_depth=max_depth, max_features=max_features,  
                                    n_estimators=75, n_jobs=-1, random_state=42)
        # выполняем перекрестную проверку        
        scores = cross_val_score(rf, X_trainval, y_trainval, scoring='roc_auc', cv=strat)
        # вычисляем среднее значение AUC перекрестной проверки 
        auc_score = np.mean(scores)
        # если получаем лучшее значение AUC, сохраняем его и параметры
        if auc_score > best_score:
            best_score = auc_score
            best_parameters = {'max_features': max_features, 'max_depth': max_depth}
# строим модель с параметрами, давшими наилучшее значение AUC на обучающей выборке, 
# и проверяем на тестовой выборке
rf_best = RandomForestClassifier(**best_parameters, n_estimators=300, random_state=42)
rf_best.fit(X_trainval, y_trainval)
test_score = roc_auc_score(y_test, rf_best.predict_proba(X_test)[:, 1])
print("Лучшее усредненное значение AUC cv: {:.2f}".format(best_score))
print("Наилучшие значения параметров: ", best_parameters)
print("AUC модели с наилучшими параметрами на тестовой выборке: {:.2f}".format(test_score))

In [ ]:
param_grid = {'max_features': [3, 4, 5, 6, 7, 8, 9, 12, 15, 18, 21], 'max_depth': [6, 8, 10, 12, 14, 16, 18, 20, 22]}

In [ ]:
rf = RandomForestClassifier(n_estimators=n_trees, random_state=42)
grid_search = GridSearchCV(rf, param_grid, n_jobs=-1, cv=strat)
grid_search.fit(X_trainval, y_trainval)
test_score = roc_auc_score(y_test, grid_search.predict(x_test))
print("AUC на тестовой выборке: {:.2f}".format(test_score))
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))

In [6]:
logreg = LogisticRegression()

In [7]:
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
logreg.score(X_test, y_test)

0.87861271676300579

In [13]:
logreg.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
# Из модуля sklearn.metrics импортируем функцию roc_auc_score
from sklearn.metrics import roc_auc_score
# Вычисляем значение AUC, передав функции roc_curve в качестве аргументов
# фактические значения зависимой переменной, записанные в y_control, 
# и вероятности положительного класса, вычисленные с помощью метода predict_proba
rf_auc = roc_auc_score(y_test, logreg.predict(X_test))
# Печатаем значение AUC
print("Значение AUC для контрольной выборки: {:.3f}".format(rf_auc))

Значение AUC для контрольной выборки: 0.500
